In [692]:
#avito
import requests
from bs4 import BeautifulSoup
from random import randint
from time import sleep
import pandas as pd  
import numpy as np
from pandas.core.common import flatten
import winsound

In [2]:
# узнаем количество страниц объявлений
url_home = 'https://www.avito.ru/moskva_i_mo/avtomobili/bmw-ASgBAgICAUTgtg3klyg?cd=1&s=104'
r = requests.get(url_home)
soup = BeautifulSoup(r.text, 'lxml')
pages = soup.find('div', class_='pagination-pages').find_all('a', class_='pagination-page')[-1].get('href')
total_pages = pages.split('=')[1]
total_pages = total_pages.split('&')[0]
total_pages = int(total_pages)+1

In [3]:
all_names = []
all_links = []
all_price_list = []
all_parameters = []
all_descriptions = []
all_adv_parameters =[]

In [4]:
# for i in range(1, total_pages):  запускается проход по всем страницам, для теста запустить 1 страницу
for i in range(1, 2):
    url = url_home + '?p=' + str(i)
    print(i)       
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    
    cars = soup.select('.snippet-link')
    
    hrefs = [x['href'] for x in cars]
    links = ['https://www.avito.ru' + x for x in hrefs]
    names = [x.get('title') for x in cars]
    
    for link in links:
        r = requests.get(link)
        soup = BeautifulSoup(r.text)
        try:
            price = soup.select('.js-item-price')[0]['content']
        except IndexError:
            price = 0
        all_price_list.append(int(price))
        try:
            parameter = soup.select('.item-params-list-item')
        except:
            parameter = ''
        all_parameters.append(parameter)
        try:
            description = soup.select('.item-description-text')
        except:
            description = ''
        all_descriptions.append(description)
        try:
            adv_parameter = soup.select('.advanced-params-param-item')
        except:
            adv_parameter = ''
        all_adv_parameters.append(adv_parameter)
            
        sleep(randint(1,3))
    all_names.append(names)
    all_links.append(links)
winsound.PlaySound('SystemExclamation', winsound.SND_ALIAS)
all_names = list(flatten(all_names))
all_links = list(flatten(all_links))

1


In [731]:
dct = {'links' : all_links, 'title' : all_names, 'price' : all_price_list, 'parameter' : all_parameters, 'description' : all_descriptions, 'adv_parameter' : all_adv_parameters}

In [732]:
pd.set_option('max_colwidth', 50)
pd.set_option('display.max_columns', 100)
df = pd.DataFrame(dct) 

In [733]:
df.drop_duplicates(subset=['links'],inplace=True)# удаляем дубликаты
df = df.drop(df[df['price']==0].index) # удаляем записи с нулевым значение цены (рекламные объявления)
df = df.drop(df[df.title.str.contains('Новый')] .index) # удаляем записи с объявлениями по новым авто
df = df.reset_index(drop='True') # обновляем индексы

In [734]:
# обработка данных в parameter
df.parameter.replace('<li class="item-params-list-item"> <span class="item-params-label">', '') # удаляем названия тегов
df.parameter = df.parameter.apply(lambda x: list(flatten(x))) # из многоуровневого списка в одноуровневый
df.parameter = df.parameter.apply(lambda x: ''.join(x)) # из списка в строку
df_sp = df['parameter'].str.split(':',expand=True)
df_work = pd.merge(df, df_sp, left_index=True, right_index=True)

In [735]:
df_work.columns = ['link', 'title', 'price', 'parameter', 'description', 'adv_parameter',
       'name', 'brand', 'model', 'model_date', 'modification', 'production_date', 'mileage',
       'condition', 'owners', 'VIN', 'body', 'doors', 'fuel', 'transmission', 'drive_type',
       'wheel', 'color', 'description1', 'place1']

In [736]:
df_work.head(5)

,link,title,price,parameter,description,adv_parameter,name,brand,model,model_date,modification,production_date,mileage,condition,owners,VIN,body,doors,fuel,transmission,drive_type,wheel,color,description1,place1
0,https://www.avito.ru/zagoryanskij/avtomobili/b...,"BMW X6, 2010, с пробегом, цена 1 050 000 руб. ...",1050000,Марка: BMW Модель: X6 Поколение: E71 (2007—...,"[[\n, [Автомобиль во владении пары муж (с 2010...","[[Климат однозонный], [Кожа], [Кожаный руль], ...",Марка,BMW Модель,X6 Поколение,E71 (2007—2012) Модификация,35i 3.0 AT (306 л.с.) Год выпуска,2010 Пробег,156406 км Состояние,не битый Владельцев по ПТС,2 VIN или номер кузова,X4XF************* Тип кузова,внедорожник Количество дверей,5 Тип двигателя,бензин Коробка передач,автомат Привод,полный Руль,левый Цвет,серый Комплектация,Базовая Место осмотра,"Московская область, городской округ Щёлково, ..."
1,https://www.avito.ru/moskva/avtomobili/bmw_5_s...,"BMW 5 серия, 2018, с пробегом, цена 2 999 000 ...",2999000,Марка: BMW Модель: 5 серия Поколение: G30/G...,"[[\n, [Продаю личный авто. Машина выходного дн...","[[Климат многозонный], [Управление на руле], [...",Марка,BMW Модель,5 серия Поколение,G30/G31 (2016—н. в.) Модификация,530i 2.0 xDrive Steptronic (249 л.с.) Год вы...,2018 Пробег,16000 км Состояние,не битый Владельцев по ПТС,1 VIN или номер кузова,X4XJ************* Тип кузова,седан Количество дверей,4 Тип двигателя,бензин Коробка передач,автомат Привод,полный Руль,левый Цвет,чёрный Комплектация,M Sport Место осмотра,"Москва, Серпуховско-Тимирязевская линия, метр..."
2,https://www.avito.ru/vidnoe/avtomobili/bmw_x7_...,"BMW X7, 2019, с пробегом, цена 5 800 000 руб. ...",5800000,Марка: BMW Модель: X7 Поколение: G07 (2018—...,"[[\n, [Автомобиль без нареканий (царапин и про...","[[Климат многозонный], [Атермальное остекление...",Марка,BMW Модель,X7 Поколение,G07 (2018—н. в.) Модификация,40i 3.0 xDrive Steptronic (340 л.с.) Год вып...,2019 Пробег,13000 км Состояние,не битый Владельцев по ПТС,1 VIN или номер кузова,X4XC************* Тип кузова,внедорожник Количество дверей,5 Тип двигателя,бензин Коробка передач,автомат Привод,полный Руль,левый Цвет,синий Комплектация,M Sport Pure Место осмотра,"Московская область, Ленинский городской округ..."
3,https://www.avito.ru/moskva/avtomobili/bmw_x6_...,"BMW X6, 2019, с пробегом, цена 4 110 000 руб. ...",4110000,Марка: BMW Модель: X6 Поколение: F16 (2014—...,"[[\n, [Покупал в Авилон BMW в декабре 2018. Ко...",[],Марка,BMW Модель,X6 Поколение,F16 (2014—н. в.) Модификация,30d 3.0 AT (249 л.с.) Год выпуска,2019 Пробег,24100 км Состояние,не битый Владельцев по ПТС,1 VIN или номер кузова,X4XK************* Тип кузова,внедорожник Количество дверей,5 Тип двигателя,дизель Коробка передач,автомат Привод,полный Руль,левый Цвет,чёрный Комплектация,Luxury Место осмотра,"Москва, Святоозёрская улица, 24"
4,https://www.avito.ru/podolsk/avtomobili/bmw_x3...,"BMW X3, 2012, с пробегом, цена 1 590 000 руб. ...",1590000,Марка: BMW Модель: X3 Поколение: F25 (2010—...,"[[\n, [Своевременно обслуживался. В салоне не ...","[[Гидроусилитель], [Кондиционер], [Кожа], [Пер...",Марка,BMW Модель,X3 Поколение,F25 (2010—2014) Модификация,20d 2.0 xDrive AT (184 л.с.) Год выпуска,2012 Пробег,25000 км Состояние,не битый Владельцев по ПТС,1 VIN или номер кузова,X4XW************* Тип кузова,внедорожник Количество дверей,5 Тип двигателя,дизель Коробка передач,автомат Привод,полный Руль,левый Цвет,серый Комплектация,Базовая Место осмотра,"Московская область, Подольск, улица Свердлова..."


In [737]:
# собираем датафрейм для модели
avito_cars = pd.DataFrame([1 for x in range(1, len(df_work)+1)], columns=['test'])
avito_cars['style'] = df_work.modification.str.split().str[0]
avito_cars['body'] = df_work.body.str.split().str[0]
avito_cars['color'] = df_work.color.str.split().str[0]
avito_cars['fuel'] = df_work.fuel.str.split().str[0]
avito_cars['model_date'] = df_work.model_date.str.split('(').str[1].str.split('—').str[0]
avito_cars['doors'] = df_work.doors.str.split().str[0]
avito_cars['production_date'] = df_work.production_date.str.split().str[0]
avito_cars['transmission'] = df_work.transmission.str.split().str[0]
avito_cars['engine_displacement'] = df_work.modification.str.split().str[1]
avito_cars['engine_power'] = df_work.modification.str.split('(').str[1].str.split(' ').str[0]
avito_cars['mileage'] = df_work.mileage.str.split().str[0]
avito_cars['drive_type'] = df_work.drive_type.str.split().str[0]
avito_cars['wheel'] = df_work.wheel.str.split().str[0]
df_work.condition = df_work.condition.apply(lambda x: x.replace('Владельцев по ПТС', ''))
avito_cars['condition'] = df_work.condition
avito_cars['owners'] = df_work.owners.str.split().str[0]
avito_cars['vehicle_title'] = np.nan
avito_cars['description'] = df_work.description1.str.split().str[0]
avito_cars['features'] = df_work.adv_parameter
avito_cars['ownership'] = np.nan
avito_cars['price'] = df_work.price
avito_cars.head(5)

,test,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,wheel,condition,owners,vehicle_title,description,features,ownership,price
0,1,35i,внедорожник,серый,бензин,2007,5,2010,автомат,3.0,306,156406,полный,левый,не битый,2,NaN,Базовая,"[[Климат однозонный], [Кожа], [Кожаный руль], ...",NaN,1050000
1,1,530i,седан,чёрный,бензин,2016,4,2018,автомат,2.0,249,16000,полный,левый,не битый,1,NaN,M,"[[Климат многозонный], [Управление на руле], [...",NaN,2999000
2,1,40i,внедорожник,синий,бензин,2018,5,2019,автомат,3.0,340,13000,полный,левый,не битый,1,NaN,M,"[[Климат многозонный], [Атермальное остекление...",NaN,5800000
3,1,30d,внедорожник,чёрный,дизель,2014,5,2019,автомат,3.0,249,24100,полный,левый,не битый,1,NaN,Luxury,[],NaN,4110000
4,1,20d,внедорожник,серый,дизель,2010,5,2012,автомат,2.0,184,25000,полный,левый,не битый,1,NaN,Базовая,"[[Гидроусилитель], [Кондиционер], [Кожа], [Пер...",NaN,1590000


In [ ]:
avito_cars.to_csv('avito_cars.csv')